This notebook is an overview and walkthough of the nascent pluripotency atlas. 

The prerequisites for creating the atlas are a) gene expression data for each sample, and b) metadata for each sample containing at a minimum the experimental platform the sample was measured on. These two files are usually called 'data' and 'annotations' respectively. 

This notebook also reads in some externally processed data, which we hope to process ourselves and include in the near future. These external datasets are recognisable by the GSE in their filename.

In [29]:
import numpy as np
import pandas as pd
import sklearn
import functions
from general_processing.process_functions import convert_symbols_to_ensembl 

Reading in expression data and metadata (annotations.tsv). For plotting, a 'display_metadata' field is required in the annotations dataframe, so I have used a temporary column here, the 'generic_sample_type'. To see the format of these dataframes just have a look at the example .tsv files I have here.

In [30]:
data        = pd.read_csv('../data/pluripotent_atlas_data.tsv', sep='\t', index_col=0)
annotations = pd.read_csv('../data/pluripotent_annotations.tsv', sep='\t', index_col=0)
data = data[annotations.index]

lizzis_anno    = pd.read_csv('/Users/pwangel/PlotlyWorkspace/combine_data/naive_stemcells/stemcell_annotations.tsv', sep='\t', index_col=0)
#lizzis_anno.index = [str(i)+";"+str(j) for i, j in zip(lizzis_anno.chip_id, lizzis_anno.Dataset.astype(int))]
annotations = annotations.merge(lizzis_anno[['LM_Group_COLOR']], how='left', left_index=True, right_index=True)
annotations.LM_Group_COLOR = annotations[['LM_Group_COLOR']].fillna("Unannotated")
annotations.index = [str(int(i))+'_'+str(j) for i,j in zip(annotations.Dataset, annotations.chip_id)]
data.columns = annotations.index

day_anno = pd.read_csv('/Users/pwangel/Downloads/pluripotent_anno_feeder_day_year.tsv', sep='\t', index_col=0)
annotations = annotations.merge(day_anno[['Day', 'Year','Feeder']], how='left', left_index=True, right_index=True)

annotations['display_metadata'] = annotations.generic_sample_type
#annotations = annotations.loc[annotations.Platform_Category!='Illumina V2']

lizzi_genes = pd.read_csv('/Users/pwangel/Downloads/early_embryo_genelists.csv', sep=',')
for i_col in lizzi_genes.columns:
    lizzi_genes[i_col] = pd.Series(convert_symbols_to_ensembl(lizzi_genes.copy().set_index(i_col)).index)

This is a temporary hack to include some externally processed data and see how the atlas would look. They look good so I recommend they be processed with the stemformatics pipeline and included. I have not placed the external data into my git repo, it would be simpler just to process with s4m pipeline.

In [31]:
df1 = pd.read_csv('/Users/pwangel/Data/External_Data/GSE114873/GSE114873_Cufflinks_output_FPKM_and_Gene_Names.txt', sep='\t', index_col=0)[['condition', 'replicate', 'raw_frags']]
df1['sample_id'] = [i+'_'+j for i,j in zip(df1.condition.to_list(),df1.replicate.to_list())]
df1.drop(labels=['condition', 'replicate'], axis=1, inplace=True)
df1=df1.pivot(columns='sample_id')
df1.columns = df1.columns.droplevel()
df1 = convert_symbols_to_ensembl(df1)

df2 = pd.read_csv('/Users/pwangel/Data/External_Data/GSE123055/GSE123055_counts.txt', sep='\t', index_col=0)
df2.drop(labels=['Unnamed: 52'], axis=1, inplace=True)

#This dataset is weird, not using it
#df3 = pd.read_csv('/Users/pwangel/Data/External_Data/GSE131551/GSE131551_human_bulk.raw_count_matrix.tsv', sep='\t', index_col=0)
#df3.drop(labels='geneID', axis=1, inplace=True)

df3 = pd.read_csv('/Users/pwangel/Data/External_Data/GSE137295/GSE137295_All_Raw_CountData.csv', index_col=0)

df4 = pd.read_csv('/Users/pwangel/Data/External_Data/GSE167089/GSE167089_counts.tsv', sep='\t',index_col=0)

As these external datasets do not have metadata, I created my own temporary external annotations dataframe.

In [32]:
ext_annotations = pd.DataFrame(index=df1.columns.to_list()+df2.columns.to_list()+df3.columns.to_list()+df4.columns.to_list())
ext_annotations['Platform_Category']='RNASeq'
ext_annotations['Dataset'] = ['GSE114873' for i in range(df1.shape[1])] + ['GSE123055' for i in range(df2.shape[1])]\
                           + ['GSE137295' for i in range(df3.shape[1])] + ['GSE167089' for i in range(df4.shape[1])]
ext_annotations['generic_sample_type'] = 'Unannotated'
ext_annotations['display_metadata']  = [str(i_dataset)+'<br>'+str(i_sample) for i_dataset, i_sample in zip(ext_annotations.Dataset.values, ext_annotations.index.values)]

Refine the data to only include those datasets appearing in the collabrorative annotations google docs file shared by the the biologists working on the project. These are the datasets of biological interest.

Also include the externally processed data in this step.

In [33]:
#This tsv has the datasets of biological interest
datasets_to_keep = pd.read_csv('../data/Pluripotency datasets in stemformatics - existing stemformatics data.tsv', sep='\t')['Dataset'].values
annotations = annotations.loc[np.in1d(annotations.Dataset, datasets_to_keep)]

annotations = pd.concat([annotations, ext_annotations], sort=True)
data = data.merge(df1, how='inner', left_index=True, right_index=True)
data = data.merge(df2, how='inner', left_index=True, right_index=True)
data = data.merge(df3, how='inner', left_index=True, right_index=True)
data = data.merge(df4, how='inner', left_index=True, right_index=True)
data = data[annotations.index]

Before we make the atlas, we will do some simple analysis on the data. In particular I would like to see the distribution of experimental platform. It would also be nice to see the cell type distributions, but without the proper metadata it is difficult

In [66]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(annotations[['Dataset', 'Platform_Category']].drop_duplicates().groupby('Platform_Category').size())

Platform_Category
Agilent            9
HG-U133_Plus_2    40
HuGene            25
Illumina V2        4
Illumina V3       15
Illumina V4       17
RNASeq            34
dtype: int64


In [67]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(annotations.groupby(['Platform_Category']).size())

Platform_Category
Agilent            418
HG-U133_Plus_2     580
HuGene             380
Illumina V2         90
Illumina V3        194
Illumina V4       1225
RNASeq             762
dtype: int64


Now to actually make the atlas. First step in the atlas two step process: transform expression values to percentile values.

In [34]:
data = functions.transform_to_percentile(data)

Second step: model the influence of platform upon expression for each gene. As this can take a while, I often save the results and just read them in rather than recompute them. In this case the results are saved in 'pluripotent_atlas_genes_with_ext.tsv'.

In [35]:
#genes = functions.calculate_platform_dependence(data, annotations)
#genes.to_csv('../data/pluripotent_atlas_genes_with_ext.tsv', sep='\t')
#genes = pd.read_csv('../data/pluripotent_atlas_genes.tsv', sep='\t')
genes = pd.read_csv('../data/pluripotent_atlas_genes_with_ext.tsv', sep='\t') 

Run the PCA on the expression data of the filtered, transformed genes. The value of the gene filter threshold is 0.25. I have not looked closely at this value. Perhaps a higher value would allow more components into the PCA. 

In [36]:
pca        = sklearn.decomposition.PCA(n_components=10, svd_solver='full')
pca.fit(functions.transform_to_percentile(data.loc[genes.Platform_VarFraction.values<=0.25]).transpose())
pca_coords = pca.transform(functions.transform_to_percentile(data.loc[genes.Platform_VarFraction.values<=0.25]).transpose())

Highlight average expression for some gene sets provided by Martin Pera (from Lizzi Masons paper).

In [46]:
for i_set in lizzi_genes.columns:
    for i_sample in annotations.index:
        annotations.loc[i_sample, i_set] = data.loc[lizzi_genes[i_set].dropna(),i_sample].mean()

/Users/pwangel/anaconda3/envs/scanpy/lib/python3.7/site-packages/pandas/core/indexing.py:961: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike



I generate a separate set of coordinates for the external data as I would like to project them by themselves. The way this works in that a list of data/annotations dataframes is passed to the plot function. The first set of data/annotations is the base and subsequent sets are projected on. The plot the pca is saved as a .html in the <out_file> location.

In [44]:
pca_coords_ext = pca.transform(functions.transform_to_percentile(data.loc[genes.Platform_VarFraction.values<=0.25][ext_annotations.index]).transpose())

#First dataframes in the list of the base coordinates, following dataframes are projected on
functions.plot_pca([pca_coords, pca_coords_ext], [annotations, ext_annotations],pca, \
                   labels=['generic_sample_type', 'Platform_Category', 'Dataset'],\
                   colour_dict={}, pcs=[1,2,3], \
                   out_file='/Users/pwangel/Downloads/pluripotent_atlas_with_external.html')

Now try to 'zoom in' on the pluripotent cells (isolate them by applying k means clustering). This is a fairly rough way to identify the samples that are relevant to the 'naive' vs 'primed' analysis. I want stem cells only, no differentiated samples, this is best cleared up by the biological annotations but k means will do for now.

In [40]:
kmeans = sklearn.cluster.KMeans(n_clusters=4).fit(pca_coords)
annotations['K Means'] = kmeans.labels_
ext_annotations['K Means'] = annotations['K Means'].loc[ext_annotations.index]

Plot the PCA again but now with the kmeans clusters, so we can identify the biology of each cluster.

In [48]:
functions.plot_pca(pca_coords, annotations,pca, \
                   labels=['generic_sample_type', 'Platform_Category', 'Dataset', 'K Means', 'LM_Group_COLOR']+lizzi_genes.columns.tolist(), \
                   colour_dict={"Unannotated":'grey'}, pcs=[1,2,3], \
                   out_file='/Users/pwangel/Downloads/pluripotent_kmeans_atlas_with_external.html')

Essentially a repeat of the previous process to generate an atlas, but now selecting only stem cells first. WARNING: the number label of the stemcells group in the kmeans clustering is random each time you run it. 

In [49]:
pluripotent_annotations = annotations.loc[(annotations['K Means']==0) | np.in1d(annotations.Dataset.values, [7253, 7240, 7124, 7135, 'GSE137295', 'GSE123055', 'GSE114873', 'GSE167089'])]
pluripotent_data = data[pluripotent_annotations.index]
#pluripotent_genes = functions.calculate_platform_dependence(pluripotent_data, pluripotent_annotations)
#pluripotent_genes.to_csv('../data/pluripotent_only_atlas_genes_with_ext.tsv', sep='\t')
pluripotent_genes = pd.read_csv('../data/pluripotent_only_atlas_genes_with_ext.tsv', sep='\t')

In [72]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pluripotent_annotations[['Dataset', 'Platform_Category']].drop_duplicates().groupby('Platform_Category').size())

Platform_Category
Agilent            9
HG-U133_Plus_2    34
HuGene            19
Illumina V2        4
Illumina V3       15
Illumina V4       12
RNASeq            27
dtype: int64


In [73]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pluripotent_annotations.groupby(['Platform_Category']).size())

Platform_Category
Agilent           253
HG-U133_Plus_2    365
HuGene            202
Illumina V2        67
Illumina V3       119
Illumina V4       616
RNASeq            438
dtype: int64


In [74]:
pluripotent_annotations.generic_sample_type.value_counts()

induced pluripotent stem cell       1059
embryonic stem cell                  372
Unannotated                          119
ESC                                  105
iPSC                                  59
ESC-derived endoderm                  50
Sample Type                           33
ESC-derived neuron                    28
pluripotent                           25
ESC-derived NP                        21
NCC                                   17
mesoderm                              14
EB                                    13
iPSC-derived endoderm                 13
endoderm                               9
cardiac cell fate commitment           9
ESC-derived FP                         9
iPSC-intermediate                      8
ESC-derived primitive streak           7
ESC-derived foregut                    6
NP                                     6
ESC-derived¬†precardiac mesoderm       6
hepatocyte                             5
EFO:0004905                            4
ESC-derived mese

In [23]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(pluripotent_annotations.groupby(['Platform_Category','generic_sample_type']).size())

Platform_Category  generic_sample_type             
Agilent            EB                                    9
                   ECC                                   2
                   ESC                                   1
                   ESC-derived endoderm                  3
                   ESC-derived primitive streak          4
                   embryonic stem cell                  43
                   iPSC-intermediate                     8
                   induced pluripotent stem cell       181
                   macrophage                            2
HG-U133_Plus_2     EB                                    4
                   ESC                                   2
                   ESC-derived NP                        6
                   ESC-derived endoderm                 18
                   ESC-derived foregut                   6
                   ESC-derived hepatocyte                4
                   ESC-derived mesendoderm               4
    

Save this subset for future fun

In [14]:
pluripotent_annotations.to_csv('../data/pluripotent_RNASeq_annotations.tsv', sep='\t')

Run the PCA again, using a threshold of 0.2. The general pattern with these thresholds is that the narrower the biological range of samples is, the stricter the threshold must be. Lower biological variation means a greater fraction of platform variation.

In [51]:
pca        = sklearn.decomposition.PCA(n_components=10, svd_solver='full')
pca.fit(functions.transform_to_percentile(pluripotent_data.loc[pluripotent_genes.Platform_VarFraction.values<=0.2]).transpose())
pca_coords = pca.transform(functions.transform_to_percentile(pluripotent_data.loc[pluripotent_genes.Platform_VarFraction.values<=0.2]).transpose())

Plot the stemcell only PCA (and save it).

In [54]:
functions.plot_pca(pca_coords, pluripotent_annotations,pca, \
                   labels=['generic_sample_type', 'Platform_Category', 'Dataset', 'LM_Group_COLOR', 'Day', 'Year', 'Feeder']+lizzi_genes.columns.tolist(), \
                   colour_dict={"Unannotated":"grey"}, pcs=[1,2,3], \
                   out_file='/Users/pwangel/Downloads/pluripotent_only_atlas_with_external.html')